# Ingest SEC DERA data into Trino pipeline

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

In [ ]:
from dotenv import dotenv_values, load_dotenv
from osc_ingest_trino import *
import os
import pathlib
import pandas as pd
import uuid
import trino
from sqlalchemy.engine import create_engine

Load Environment Variables

In [ ]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [ ]:
import boto3

s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)

In [ ]:
s3 = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ["S3_DEV_ENDPOINT"],
    aws_access_key_id=os.environ["S3_DEV_ACCESS_KEY"],
    aws_secret_access_key=os.environ["S3_DEV_SECRET_KEY"],
)
bucket = s3.Bucket(os.environ["S3_DEV_BUCKET"])
bucket.name

Create custom meta data and declare variable for schema and table for the data set

In [ ]:
sub_meta_content = {
    'dataset_key': 'SEC-DERA-SUB',
    'title': 'SEC DERA Submission Dataset',
    'description': 
    '''The DERA Financial Statement Data Sets provide numeric information from the face financials of all financial statements.
    
    This data is extracted from exhibits to corporate financial reports filed with the Commission using eXtensible Business Reporting Language (XBRL).  As compared to the more extensive Financial Statement and Notes Data Sets, which provide the numeric and narrative disclosures from all financial statements and their notes, the Financial Statement Data Sets are more compact.''',
    'version': '2020q4',
    'release_date': '20201231',
    'fields': [
    {
        'adsh':'Accession Number. The 20-character string formed from the 18-digit number assigned by the SEC to each EDGAR submission.',
        'cik':'Central Index Key (CIK). Ten digit number assigned by the SEC to each registrant that submits filings.',
        'name':'Name of registrant. This corresponds to the name of the legal entity as recorded in EDGAR as of the filing date.',
        'sic':'Standard Industrial Classification (SIC). Four digit code assigned by the SEC as of the filing date, indicating the registrant’s type of business.',
        'countryba':'The ISO 3166-1 country of the registrant’s business address.',
        'stprba':'The state or province of the registrant’s business address, if field countryba is US or CA.',
        'cityba':'The city of the registrant’s business address.',
        'zipba':'The zip code of the registrant’s business address.',
        'bas1':'The first line of the street of the registrant’s business address.',
        'bas2':'The second line of the street of the registrant’s business address.',
        'baph':'The phone number of the registrant’s business address.',
        'countryma':'The ISO 3166-1 country of the registrant’s mailing address.',
        'stprma':'The state or province of the registrant’s mailing address, if field countryma is US or CA.',
        'cityma':'The city of the registrant’s mailing address.',
        'zipma':'The zip code of the registrant’s mailing address.',
        'mas1':'The first line of the street of the registrant’s mailing address.',
        'mas2':'The second line of the street of the registrant’s mailing address.',
        'countryinc':'The country of incorporation for the registrant.',
        'stprinc':'The state or province of incorporation for the registrant, if countryinc is US or CA.',
        'ein':'Employee Identification Number, 9 digit identification number assigned by the Internal Revenue Service to business entities operating in the United States.',
        'former':'Most recent former name of the registrant, if any.',
        'changed':'Date of change from the former name, if any.',
        'afs':'Filer status with the SEC at the time of submission:\n\
1-LAF=Large Accelerated,\n\
2-ACC=Accelerated,\n\
3-SRA=Smaller Reporting Accelerated,\n\
4-NON=Non-Accelerated,\n\
5-SML=Smaller Reporting Filer,\n\
NULL=not assigned.',
        'wksi':'Well Known Seasoned Issuer (WKSI). An issuer that meets specific SEC requirements at some point during a 60-day period preceding the date the issuer satisfies its obligation to update its shelf registration statement.',
        'fye':'Fiscal Year End Date, rounded to nearest month-end.',
        'form':'The submission type of the registrant’s filing.',
        'period':'Balance Sheet Date, rounded to nearest month-end.',
        'fy':'Fiscal Year Focus (as defined in EFM Ch. 6).',
        'fp':'Fiscal Period Focus (as defined in EFM Ch. 6) within Fiscal Year. The 10-Q for the 1st, 2nd and 3rd quarters would have a fiscal period focus of Q1, Q2 (or H1), and Q3 (or M9) respectively, and a 10-K would have a fiscal period focus of FY.',
        'filed':'The date of the registrant’s filing with the Commission.',
        'accepted':'The acceptance date and time of the registrant’s filing with the Commission. Filings accepted after 5:30pm EST are considered filed on the following business day.',
        'prevrpt':'Previous Report –TRUE indicates that the submission information was subsequently amended.',
        'detail':'TRUE indicates that the XBRL submission contains quantitative disclosures within the footnotes and schedules at the required detail level (e.g., each amount).',
        'instance':'The name of the submitted XBRL Instance Document (EX-101.INS) type data file. The name often begins with the company ticker symbol.',
        'nciks':'Number of Central Index Keys (CIK) of registrants (i.e., business units) included in the consolidating entity’s submitted filing.',
        'aciks':'Additional CIKs of co-registrants included in  a consolidating entity’s EDGAR submission, separated by spaces. If there are no other co-registrants (i.e., nciks=1), the value of aciks is NULL.  For a very small number of filers, the list of co-registrants is too long to fit in the field.  Where this is the case, PARTIAL will appear at the end of the list indicating that not all co-registrants’ CIKs are included in the field; users should refer to the complete submission file for all CIK information.'
    }]
}
num_meta_content = {
    'dataset_key': 'SEC-DERA-NUM',
    'title': 'SEC DERA Number Dataset',
    'description': 
    '''The DERA Financial Statement Data Sets provide numeric information from the face financials of all financial statements.
    
    This data is extracted from exhibits to corporate financial reports filed with the Commission using eXtensible Business Reporting Language (XBRL).  As compared to the more extensive Financial Statement and Notes Data Sets, which provide the numeric and narrative disclosures from all financial statements and their notes, the Financial Statement Data Sets are more compact.''',
    'version': '2020q4',
    'release_date': '20201231',
    'fields': [
    {
        'adsh':'Accession Number. The 20-character string formed from the 18-digit number assigned by the SEC to each EDGAR submission.',
        'tag':'The unique identifier (name) for a tag in a specific taxonomy release.',
        'version':'For a standard tag, an identifier for the taxonomy; otherwise the accession number where the tag was defined.',
        'coreg':'If specified, indicates a specific co-registrant, the parent company, or other entity (e.g., guarantor). NULL indicates the consolidated entity.',
        'ddate':'The end date for the data value, rounded to the nearest month end.',
        'qtrs':'The count of the number of quarters represented by the data value, rounded to the nearest whole number. “0” indicates it is a point-in-time value.',
        'uom':'The unit of measure for the value.',
        'value':'The value. This is not scaled, it is as found in the Interactive Data file, but is limited to four digits to the right of the decimal point.',
        'footnote':'The text of any superscripted footnotes on the value, as shown on the statement page, truncated to 512 characters, or if there is no footnote, then this field will be blank.'
    }]
}
schemaname = 'sec_dera'

Convert custom metadata content in json format into Pandas DataFrame

In [ ]:
df_meta_fields = pd.json_normalize(sub_meta_content, record_path =['fields'], meta=['dataset_key']).convert_dtypes()
df_meta_fields

In [ ]:
df_meta_fields.info(verbose=True)

In [ ]:
df_meta_table = pd.json_normalize(sub_meta_content, max_level=0)
df_meta_table.drop('fields', inplace=True, axis=1)
df_meta_table['schema'] = schemaname
df_meta_table = df_meta_table.convert_dtypes()
df_meta_table

In [ ]:
df_meta_table.info(verbose=True)

Open a Trino connection using JWT for authentication

In [ ]:
sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
print("connecting with engine " + str(engine))
connection = engine.connect()

In [ ]:
catalogname = 'osc_datacommons_dev'

# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {catalogname}')
for row in schema_read.fetchall():
    print(row)

Create ingestion schema based on source data name and remove old tables if necessary

In [ ]:
schema_check = engine.execute(f'create schema if not exists {catalogname}.{schemaname}')
for row in schema_check.fetchall():
    print(row)

In [ ]:
engine.execute(f'show tables in {catalogname}.{schemaname}').fetchall()

Prepare GLEIF matching data

In [ ]:
gleif_file = s3_resource.Object(os.environ['S3_LANDING_BUCKET'],f'mtiemann-GLEIF/DERA-matches.csv')
gleif_file.download_file(f'/tmp/dera-gleif.csv')
gleif_df = pd.read_csv(f'/tmp/dera-gleif.csv', header=0, sep=',', dtype=str, engine='c')
gleif_dict = { k:v for k, v in zip(gleif_df.name, gleif_df.LEI) }

Load the SUB and NUM tables into Trino

In [ ]:
import re
import io

# Add a unique identifier to the data set
uid = str(uuid.uuid4())

dera_regex = re.compile(r' ?/.*$')

for tablename in [ 'sub', 'num' ]:
    src_file = s3_resource.Object(os.environ['S3_LANDING_BUCKET'],f'SEC-DERA/2020q4/{tablename}.txt')
    timestamp = src_file.last_modified.isoformat()
    src_file.download_file(f'/tmp/dera-{tablename}-{timestamp}.csv')
    df = pd.read_csv(f'/tmp/dera-{tablename}-{timestamp}.csv', header=0, sep='\t', dtype=str, nrows = 10, engine='c')

    df['uuid'] = uid
    # Print the output
    
    if tablename=='sub':
        df.name = df.name.map(lambda x: re.sub(dera_regex, '', x))
        df['LEI'] = df.name.map(gleif_dict)
        df.cik = df.cik.astype('int32')
        df.sic = df.sic.astype('int32')
        df.ein = df.ein.astype('int64')
        # df.wksi = df.wksi.astype('bool')
        df.wksi = df.wksi.astype('int32')
        df.period = pd.to_datetime(df.period, format='%Y%m%d')
        df.fy = pd.to_datetime(df.fy, format='%Y')
        df.filed = pd.to_datetime(df.filed, format='%Y%m%d')
        df.accepted = pd.to_datetime(df.accepted, format='%Y-%m-%d %H:%M:%S')
        # df.prevrpt = df.prevrpt.astype('bool')
        # df.detail = df.detail.astype('bool')
        df.prevrpt = df.prevrpt.astype('int32')
        df.detail = df.detail.astype('int32')
        df.nciks = df.nciks.astype('int32')
        
        cols = df.columns.tolist()
        cols = cols[0:3] + [cols[-1]] + cols[3:-1]
        df = df[cols]
    elif tablename=='num':
        # documentation wrongly lists coreg as NUMERIC length 256.  It is ALPHANUMERIC.
        df.ddate = pd.to_datetime(df.ddate, format='%Y%m%d')
        df.qtrs = df.qtrs.astype('int32')
        df.value = df.value.astype('float64')
    print(df.dtypes)
    display(df.head())

    buf = io.BytesIO()
    df.to_parquet(path=buf)
    buf.seek(0)
    bucket.upload_fileobj(Fileobj=buf,
                          Key=f'trino/{schemaname}/{tablename}/data.parquet')

    table_check = engine.execute(f'drop table if exists {catalogname}.{schemaname}.{tablename}')
    for row in table_check.fetchall():
        print(row)

    schema = create_table_schema_pairs(df)
    tabledef = f"""
    create table if not exists {catalogname}.{schemaname}.{tablename} (
    {schema}
    ) with (
        format = 'parquet',
        external_location = 's3a://{bucket.name}/trino/{schemaname}/{tablename}/'
    )
    """
    print(tabledef)

    table_create = engine.execute(tabledef)
    for row in table_create.fetchall():
        print(row)

    dataset_query = ('SELECT * FROM {cname}.{sname}.{tname} limit 10').format(cname=catalogname,sname=schemaname,tname=tablename)
    print(dataset_query)
    dataset = engine.execute(dataset_query)
    for row in dataset.fetchall():
        print(row)